In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
from sympy import *
from IPython.display import display
import time
import pickle
import datetime
%env USE_SYMENGINE 1

epsilon, epsiloninv, u, v, a, theta = symbols('\\epsilon, \\eta, u, v, a, \\theta', real=True)


q = symbols('q', real=False)
q_conj = symbols('\\tilde{q}', real=False)


W_plus = Symbol('W_+', real=True)
W_minus = Symbol('W_-', real=True)
w_plus = Symbol('w_+', real=True)
w_minus = Symbol('w_-', real=True)
V_sym = Symbol('V')
N_sym = Symbol('N')
A_sym = Symbol('A')
phi_sym = Symbol('\\phi')
n_sym = Symbol('n')
a_sym = Symbol('a')

differentiable_symbols = [u, v, a, theta, q, q_conj, w_plus, w_minus, W_plus, W_minus, V_sym, N_sym, A_sym, phi_sym, n_sym, a_sym]


env: USE_SYMENGINE=1


In [3]:

def var_deriv_name(var):
    if "_{x" in var.name:
        return var.name[:-1] + 'x}'
    else:
        return '(' + var.name + ')_{x}'
        #return var.name + '_x'
        
def get_var_name_from_deriv(sym):
    start = sym.name.find('(')
    end = sym.name.find(')')
    if start != -1 and end != -1:
        sym_name = sym.name[start+1:end]
        return sym_name
        #orders.append(len(sym.name) - 4 - sym.name.find(')_{'))
    else:
        sym_name = sym.name
        return sym_name
        #orders.append(0)

def get_order_from_deriv(sym):
    start = sym.name.find('(')
    end = sym.name.find(')')
    if start != -1 and end != -1:
        sym_name = sym.name[start+1:end]
        return len(sym.name) - 4 - sym.name.find(')_{')
    else:
        return 0
        
def deriv(poly, append_to_list=True):
    res = 0
    original = differentiable_symbols.copy()
    for sym in original:
        deriv_term = Derivative(poly, sym).doit()
        if deriv_term != 0:
            #print("looking at: ", sym)
            newName = var_deriv_name(sym)

            if newName in [s.name for s in differentiable_symbols]:
                dsym = differentiable_symbols[[s.name for s in differentiable_symbols].index(newName)]
                #print("newName found: ", dsym)
            else:
                dsym = Symbol(newName, real=True)
                if append_to_list:
                    print("new differentiable symbol: ", dsym)
                    differentiable_symbols.append(dsym)
                #print("newName not found: ", dsym)
            #print("syms: ", syms)
            res += deriv_term * dsym
            #print("res: ", res)
    
    return res
    

def higher_deriv(var, n):
    if n > 0:
        return deriv(higher_deriv(var, n-1))
    else:
        return var
    
def variation(expr, sym):
    if not hasattr(expr, 'free_symbols'):
        return 0
    
    res = 0
    order = 0

    syms = []
    orders = []

    start = sym.name.find('(')
    end = sym.name.find(')')
    if start != -1 and end != -1:
        sym_name = sym.name[start+1:end]
        syms.append(sym)
        orders.append(len(sym.name) - 4 - sym.name.find(')_{'))
    else:
        sym_name = sym.name
        syms.append(sym)
        orders.append(0)

    for s in expr.free_symbols:
        start = s.name.find('(')
        end = s.name.find(')')
        if start != -1 and end != -1 and sym_name == s.name[start+1:end]:
            if s.name not in [sym.name for sym in syms]:
                syms.append(s)
                orders.append(len(s.name) - 4 - s.name.find(')_{'))    
    
    for (sym, order) in zip(syms, orders):
        res += (-1)**order * higher_deriv(Derivative(expr, sym).doit(), order)

    return simplify(res)

def polynomize(expr):
    return simplify(Poly(expr, epsilon, epsiloninv).subs(epsiloninv, 1/epsilon))

def depolynomize(poly):
    monoms = poly.monoms()
    coeffs = poly.coeffs()
    
    X = 0
    for (k, m) in enumerate(monoms):
        X += epsilon**m[0] * epsiloninv**m[1] * coeffs[k]
            
    return X

def poly_simplify(expr):
    return depolynomize(polynomize(expr))

def substituter(expr, var, sub, magnitude=1, scale=1):
    if not hasattr(expr, 'subs'):
        return expr
    for subvar in sub.free_symbols:
        if var in differentiable_symbols and subvar not in differentiable_symbols:
            print("Warning: Substituting differentiable by non-differentiable symbol.")
            #differentiable_symbols.append(subvar)
            pass
        
    original = differentiable_symbols.copy()
    expr = expr.subs(var, magnitude * sub)
    
    cont = True
    m = 0
    while cont:
        #print("var", var, "sub", sub)
        dvar = deriv(var, append_to_list=False)
        m += 1
        
        if dvar in differentiable_symbols:
            dsub = simplify(deriv(sub))
            #print("new sub variable from", sub, "to", dsub)
            expr = simplify(expr.subs(dvar, magnitude * scale**m * dsub))
                    
            var = dvar
            sub = dsub
                    
            #print("substited", var, "by", sub)
        else: 
            cont = False
            
    if hasattr(expr, 'applyfunc'):
        return expr.applyfunc(poly_simplify)
    else:
        return poly_simplify(expr)
    
def multi_substituter(expr, data, magnitude=1, scale=1):
    for (var, sub) in data:
        expr = substituter(expr, var, sub, magnitude=magnitude, scale=scale)
    return expr

def extract_deriv(expr, k): 
    expr = simplify(expr).expand()
    #print("expr =", expr) 
    res = 0
    if isinstance(expr, Add):
        for mon in expr.args:
            #print("mon =", mon)
            count = 0
            if hasattr(mon, "name"):
                #print(mon.name)
                if "_{x" in mon.name:
                    count += 1
                    #print("count =", count)
            else:
                for fac in mon.args:
                    #print("fac =", fac)
                    if hasattr(fac, "name"):
                        #print(fac.name)
                        if "_{x" in fac.name:
                            count += 1
                            #print("count =", count)
                    elif isinstance(fac, Pow):
                        #print(fac.base.name, fac.exp)
                        if "_{x" in fac.base.name and fac.exp > 0:
                            count += fac.exp     
                            #print("count =", count)
            #print("final count =", count)
            if count == k:
                res += mon

    else:
        mon = expr
        #print("mon =", mon)
        count = 0
        if hasattr(mon, "name"):
            #print(mon.name)
            if "_{x" in mon.name:
                count += 1
                #print("count =", count)
        elif hasattr(mon, "args"):
            for fac in mon.args:
                #print("fac =", fac)
                if hasattr(fac, "name"):
                    #print(fac.name)
                    if "_{x" in fac.name:
                        count += 1
                        #print("count =", count)
                elif isinstance(fac, Pow):
                    #print(fac.base.name, fac.exp)
                    if "_{x" in fac.base.name and fac.exp > 0:
                        count += fac.exp     
                        #print("count =", count)
        #print("final count =", count)
        if count == k:
            res += mon

    #print("res =", res)
    return res
        
def extract_deriv_alt(expr, k):
    expr = simplify(expr).expand()
    #print("expr =", expr) 
    res = 0
    if isinstance(expr, Add):
        for mon in expr.args:
            presence_deriv_q_conj = False
            #print("mon =", mon)
            count = 0
            if hasattr(mon, "name"):
                #print(mon.name)
                if "_{x" in mon.name:
                    count += 1
                    if get_var_name_from_deriv(mon) == q_conj.name:
                        presence_deriv_q_conj = True
            else:
                for fac in mon.args:
                    #print("fac =", fac)
                    if hasattr(fac, "name"):
                        #print(fac.name)
                        if "_{x" in fac.name:
                            count += 1
                            if get_var_name_from_deriv(fac) == q_conj.name:
                                presence_deriv_q_conj = True
                            #print("count =", count)
                    elif isinstance(fac, Pow):
                        #print(fac.base.name, fac.exp)
                        if "_{x" in fac.base.name and fac.exp > 0:
                            count += fac.exp     
                            if get_var_name_from_deriv(fac.base) == q_conj.name:
                                presence_deriv_q_conj = True
                            #print("count =", count)
            #print("final count =", count)
            if count == k and presence_deriv_q_conj:
                res += mon

    else:
        presence_deriv_q_conj = False
        mon = expr
        #print("mon =", mon)
        count = 0
        if hasattr(mon, "name"):
            #print(mon.name)
            if "_{x" in mon.name:
                count += 1
                if get_var_name_from_deriv(mon) == q_conj.name:
                    presence_deriv_q_conj = True
                #print("count =", count)
        elif hasattr(mon, "args"):
            for fac in mon.args:
                #print("fac =", fac)
                if hasattr(fac, "name"):
                    #print(fac.name)
                    if "_{x" in fac.name:
                        count += 1
                        if get_var_name_from_deriv(fac) == q_conj.name:
                            presence_deriv_q_conj = True
                        #print("count =", count)
                elif isinstance(fac, Pow):
                    #print(fac.base.name, fac.exp)
                    if "_{x" in fac.base.name and fac.exp > 0:
                        count += fac.exp   
                        if get_var_name_from_deriv(fac.base) == q_conj.name:
                            presence_deriv_q_conj = True
                        #print("count =", count)
        #print("final count =", count)
        if count == k and presence_deriv_q_conj:
            res += mon

    #print("res =", res)
    return res

N = 7

In [ ]:
def get_expansion(eqn, x0, var, num):
    ser = series(eqn, x0=0, x=var, n=num).removeO()
    return [simplify(ser.coeff(var**n) if n > 0 else ser.subs(var, 0)) for n in range(0, num)]

c = Symbol('c', real=True)
lam = Symbol('\\lambda')
kappa = Symbol('\\kappa')
X = Symbol('X')

part = 2 * epsilon**2 * kappa**2 * sech(X)**2
Soliton_plus = - Rational(1, 2) / epsilon**2 * lam * part / (1 - part) + (sqrt(1 - part) - 1) / epsilon**2
Soliton_minus = - Rational(1, 2) / epsilon**2 * lam * part / (1 - part) - (sqrt(1 - part) - 1) / epsilon**2

display(Soliton_plus)

print("A")
ser = series(Soliton_plus, x0=0, x=epsilon, n=12)
display(simplify(ser))
print("B")
ser = series(Soliton_minus, x0=0, x=epsilon, n=12)
display(simplify(ser))

lam = sqrt(1 - 2 * epsilon**2 * kappa**2)
part = 2 * epsilon**2 * kappa**2 * sech(X)**2
Soliton_plus = - Rational(1, 2) / epsilon**2 * lam * part / (1 - part) + (sqrt(1 - part) - 1) / epsilon**2
Soliton_minus = - Rational(1, 2) / epsilon**2 * lam * part / (1 - part) - (sqrt(1 - part) - 1) / epsilon**2

print("C")
ser = series(simplify(Soliton_plus), x0=0, x=epsilon, n=12)
display(ser)
print("D")
ser = series(simplify(Soliton_minus), x0=0, x=epsilon, n=12)
display(ser)

-\kappa**2*\lambda*sech(X)**2/(-2*\epsilon**2*\kappa**2*sech(X)**2 + 1) + (sqrt(-2*\epsilon**2*\kappa**2*sech(X)**2 + 1) - 1)/\epsilon**2

A


-\kappa**2*sech(X)**2 - \kappa**2*\lambda*sech(X)**2 - \epsilon**2*\kappa**4*(4*\lambda + 1)*sech(X)**4/2 - \epsilon**4*\kappa**6*(8*\lambda + 1)*sech(X)**6/2 - \epsilon**6*\kappa**8*(64*\lambda + 5)*sech(X)**8/8 - \epsilon**8*\kappa**10*(128*\lambda + 7)*sech(X)**10/8 - \epsilon**10*\kappa**12*(512*\lambda + 21)*sech(X)**12/16 + O(\epsilon**12)

B


\kappa**2*sech(X)**2 - \kappa**2*\lambda*sech(X)**2 - \epsilon**2*\kappa**4*(4*\lambda - 1)*sech(X)**4/2 - \epsilon**4*\kappa**6*(8*\lambda - 1)*sech(X)**6/2 - \epsilon**6*\kappa**8*(64*\lambda - 5)*sech(X)**8/8 - \epsilon**8*\kappa**10*(128*\lambda - 7)*sech(X)**10/8 - \epsilon**10*\kappa**12*(512*\lambda - 21)*sech(X)**12/16 + O(\epsilon**12)

C


-2*\kappa**2*sech(X)**2 + \epsilon**2*(-5*\kappa**4*sech(X)**4/2 + \kappa**4*sech(X)**2) + \epsilon**4*(-9*\kappa**6*sech(X)**6/2 + 2*\kappa**6*sech(X)**4 + \kappa**6*sech(X)**2/2) + \epsilon**6*(-69*\kappa**8*sech(X)**8/8 + 4*\kappa**8*sech(X)**6 + \kappa**8*sech(X)**4 + \kappa**8*sech(X)**2/2) + \epsilon**8*(-135*\kappa**10*sech(X)**10/8 + 8*\kappa**10*sech(X)**8 + 2*\kappa**10*sech(X)**6 + \kappa**10*sech(X)**4 + 5*\kappa**10*sech(X)**2/8) + \epsilon**10*(-533*\kappa**12*sech(X)**12/16 + 16*\kappa**12*sech(X)**10 + 4*\kappa**12*sech(X)**8 + 2*\kappa**12*sech(X)**6 + 5*\kappa**12*sech(X)**4/4 + 7*\kappa**12*sech(X)**2/8) + O(\epsilon**12)

D
